# Dose-Response Analysis

This notebook demonstrates dose-response curve fitting using the 4-parameter logistic (Hill) equation.

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from notebook_utils import get_api_client, demo_mode_banner

# Optional API integration (not required for the static demo below)
api_url = os.environ.get("API_URL") or os.environ.get("AMPRENTA_API_URL")
client, demo_mode = get_api_client(api_url=api_url)
if demo_mode or client is None:
    demo_mode_banner()

# Set style
plt.style.use('seaborn-v0_8')


## Example Dose-Response Data

In [ ]:
# Example dose-response data (concentrations in nM, response as % inhibition)
concentrations = np.array([0.1, 1, 10, 100, 1000, 10000, 100000])  # nM
response = np.array([5, 15, 35, 65, 85, 92, 95])  # % inhibition

df = pd.DataFrame({
    'concentration_nM': concentrations,
    'response_percent': response
})

print(df)
print(f"\nData range: {concentrations.min():.1f} - {concentrations.max():.1f} nM")

## 4-Parameter Logistic (Hill) Curve Fitting

In [ ]:
def hill_equation(x, bottom, top, ec50, hill_slope):
    """4-parameter logistic (Hill) equation.

    Parameters:
    - bottom: minimum response (baseline)
    - top: maximum response (plateau)
    - ec50: concentration at 50% response
    - hill_slope: Hill coefficient (steepness)
    """
    return bottom + (top - bottom) / (1 + (ec50 / x) ** hill_slope)

# Initial parameter guesses
initial_guess = [
    response.min(),  # bottom
    response.max(),  # top
    np.median(concentrations),  # ec50
    1.0  # hill_slope
]

# Fit the curve
popt, pcov = curve_fit(
    hill_equation,
    concentrations,
    response,
    p0=initial_guess,
    maxfev=10000
)

bottom, top, ec50, hill_slope = popt

print(f"Fitted parameters:")
print(f"  Bottom: {bottom:.2f}%")
print(f"  Top: {top:.2f}%")
print(f"  EC50: {ec50:.2f} nM")
print(f"  Hill slope: {hill_slope:.2f}")

## Calculate IC50/EC50

In [ ]:
# IC50 is the concentration at 50% inhibition
# For inhibition assays, IC50 = EC50 when baseline is 0 and top is 100
ic50 = ec50

print(f"IC50: {ic50:.2f} nM")
print(f"IC50: {ic50/1000:.4f} µM")
print(f"IC50: {ic50/1000000:.6f} mM")

## Plot Dose-Response Curve

In [ ]:
# Generate smooth curve for plotting
x_fit = np.logspace(
    np.log10(concentrations.min()),
    np.log10(concentrations.max()),
    1000
)
y_fit = hill_equation(x_fit, bottom, top, ec50, hill_slope)

# Plot
plt.figure(figsize=(10, 6))
plt.semilogx(concentrations, response, 'o', markersize=8, label='Data', color='blue')
plt.semilogx(x_fit, y_fit, '-', label='Fitted curve', color='red', linewidth=2)

# Mark IC50
plt.axvline(ic50, color='green', linestyle='--', linewidth=2, label=f'IC50 = {ic50:.2f} nM')
plt.axhline(50, color='gray', linestyle=':', alpha=0.5)

plt.xlabel('Concentration (nM)', fontsize=12)
plt.ylabel('Response (% inhibition)', fontsize=12)
plt.title('Dose-Response Curve', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()